In [1]:
!pip install interface_meta --no-index --find-links=file:../input/e2eml-inc-dependencies/interface_meta-1.2.4-py2.py3-none-any.whl
!pip install astor --no-index --find-links=file:../input/e2eml-inc-dependencies/astor-0.8.1-py2.py3-none-any.whl
!pip install formulaic --no-index --find-links=file:../input/formulaic/formulaic-0.2.4-py3-none-any.whl
!pip install autograd --no-index --find-links=file:../input/e2eml-inc-dependencies/autograd-1.3-py3-none-any.whl
!pip install autograd-gamma --no-index --find-links=file:../input/e2eml-inc-dependencies/autograd-gamma-0.5.0/dist/autograd-gamma-0.5.0.tar
!pip install lifelines --no-index --find-links=file:../input/lifelines/lifelines-0.26.4-py3-none-any.whl
!pip install ngboost --no-index --find-links=file:../input/e2eml-inc-dependencies/ngboost-0.3.12-py3-none-any.whl

In [2]:
!pip install boostaroota --no-index --find-links=file:../input/e2eml-inc-dependencies/boostaroota-1.3-py2.py3-none-any.whl
!pip install matplotlib --no-index --find-links=file:../input/e2eml-inc-dependencies/matplotlib-3.1.3-cp38-cp38-manylinux1_x86_64.whl

In [3]:
!pip install catboost --no-index --find-links=file:../input/e2eml-inc-dependencies/catboost-0.21-cp38-none-manylinux1_x86_64.whl

In [4]:
!pip install pytorch_tabnet --no-index --find-links=file:../input/e2eml-inc-dependencies/pytorch_tabnet-3.1.1-py3-none-any.whl

In [5]:
!pip install shap --no-index --find-links=file:../input/e2eml-inc-dependencies/shap-0.39.0-cp35-cp35m-linux_armv6l.whl

In [6]:
!pip install e2eml --no-index --no-dependencies --find-links=file:../input/e2eml-inc-dependencies/e2eml-2.10.4-py3-none-any.whl

In [7]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
# load libraries
from e2eml.regression import regression_blueprints
from e2eml.full_processing import postprocessing
from e2eml.full_processing.postprocessing import save_to_production, load_for_production
import gc

In [14]:
target = "score"

In [15]:
df_train = pd.read_csv("../input/jigsaw-toxic-severity-rating/comments_to_score.csv")
df_train = df_train.rename(columns={"text": "comment_text"})

In [16]:
df_train

In [17]:
batch_size = 2500
max_len = len(df_train.index)

if max_len%batch_size == 0:
    nb_batches = int(max_len/batch_size)
else:
    nb_batches = int(max_len/batch_size + 1)

In [20]:
results = []
already_batched = 0

for batch in range(nb_batches):
    start_idx = already_batched
    end_idx = already_batched + batch_size
    if max_len - already_batched < batch_size:
        batch_size = max_len-already_batched
        end_idx = already_batched + batch_size
    
    temp_df = df_train[["comment_text"]].iloc[start_idx:end_idx, :].copy()
    toxic_ml = load_for_production(file_path='../input/toxic-auto-ml/', file_name='toxic_ml_sgd_instance.dat')
    toxic_ml.ml_bp20_regression_full_processing_sgd(temp_df)
    val_y_hat = toxic_ml.predicted_values['sgd']
    results.append(pd.Series(val_y_hat))
    already_batched += batch_size
    del temp_df
    del toxic_ml
    del val_y_hat
    _ = gc.collect()

In [21]:
res_ser = pd.concat(results)
df_train["score"] = res_ser.values
df_train = df_train[['comment_id','score']]
df_train.to_csv('./submission.csv', index=False)
df_train